In [1]:
from sklearn.metrics.pairwise import cosine_similarity
import matplotlib.pyplot as plt
import scipy as sp
import scipy.stats
import pandas as pd
import numpy as np
import random
import pymysql
import datetime

In [2]:
# Mysql에서 데이터 가져오는 함수
def get_mysql_data(str):
    conn = pymysql.connect(host = "localhost",
             user= "root", password = "1234", charset = "utf8")
    cursor = conn.cursor()
    cursor.execute(str)
    data = cursor.fetchall()
    return data

In [3]:
# 부여된 가중치를 활용하여 표본을 n번 추출
def Random_Weighted_Sampling(menu_df,user_df,n):
    prs_data = []
    for i in range(len(menu_df)):
        prs_data.append(menu_df.iloc[i,:].tolist() + random.choices(range(9,15), k=2) + random.choices(range(5,11), k=2) + random.choices(range(1,7), k=1))
    df = pd.DataFrame(columns = ['menu_id','menu_name','restaurant_id','url','1','2','3','4','5'], data = prs_data)
    df_n = len(df)
    user_n = len(user_df)
    id_data = [0 for i in range(user_n)]
    user_df['count']=id_data
    rws_data = []
    #db연결
    conn = pymysql.connect(host = "localhost",
    user= "root", password = "1234", charset = "utf8")
    cursor = conn.cursor()
    #계산
    for i in range(n):
        # 무작위로 메뉴 선택
        random_index = random.randrange(df_n)
        user_index = random.randrange(user_n)
        # 메뉴의 정보
        menu_id = df.iloc[random_index].loc['menu_id'].tolist()
        time = datetime.datetime.now()
        user_name = user_df.iloc[user_index].loc['user_name']
        number = i+1
        restaurant_id = df.iloc[random_index].loc['restaurant_id']
        url = df.iloc[random_index].loc['url']
        # 메뉴의 가중치
        weight2 = df.iloc[random_index,[4,8]].to_numpy()
        weightT = df.iloc[random_index,[4,5,6,8]].to_numpy()
        weight4 = df.iloc[random_index,[4,6,7,8]].to_numpy()
        weight5 = df.iloc[random_index,[4,5,6,7,8]].to_numpy()
        # 가중치에 따른 확률 계산
        percent2 = list(weight2/np.sum(weight2))
        percentT = list(weightT/np.sum(weightT))
        percent4 = list(weight4/np.sum(weight4))
        percent5 = list(weight5/np.sum(weight5))
        # 확률에 맞게 추출
        flavor = np.random.choice([1,2,3,4],p=percentT)
        cleaning = np.random.choice([None,1,2,3],p=percent4)
        service = np.random.choice([None,1,2,3],p=percent4)
        plating = np.random.choice([None,1,2,3],p=percent4)
        price = np.random.choice([None,1,2,3,4],p=percent5)        
        couple = np.random.choice([None,0],p=percent2)        
        family = np.random.choice([None,1],p=percent2)        
        solo = np.random.choice([None,2],p=percent2)        
        drink = np.random.choice([None,3],p=percent2)        
        friend = np.random.choice([None,4],p=percent2)        
        ##유저가 평가한 횟수 카운팅
        user_df.at[user_index,'count'] = user_df.iloc[user_index].loc['count'].tolist()+1
        id = user_df.iloc[user_index].loc['count']
        
        #db insert
        #cursor.execute('''INSERT INTO young_cheline.evaluate (menu_id, time, user_name, cleaning, couple, drink, family, friend, number, 
        #        plating, price, restaurant_id, service, solo, flavor, url, id) VALUE(%s,%s,%s,%s,%s,%s,%s,%s,%s,%s,%s,%s,%s,%s,%s,%s,%s);'''
        #      ,(menu_id,time,user_name,cleaning,couple,drink,family,friend,number,plating,price,restaurant_id,service,solo,flavor,url,id))
        #conn.commit()
        
        # 모두 엮어서 하나의 데이터프레임으로 만들기
        rws_data.append([menu_id,time,user_name,cleaning,couple,drink,family,friend,number,plating,price,restaurant_id,service,solo,flavor,url,id])
        
    result_df = pd.DataFrame(columns = ['menu_id','time','user_name','cleaning','couple','drink','family','friend','number','plating','price','restaurant_id','service','solo','flavor','url','id'],data = rws_data)
    return result_df

In [4]:
menu_df = pd.DataFrame(data = get_mysql_data('''select * from young_cheline.menu'''))

In [5]:
user_df = pd.DataFrame(columns = ['id','active','email','password','role','temp_pw','time','user_name'], data = get_mysql_data('SELECT * FROM young_cheline.user'))
user_df = user_df[user_df['user_name'].notnull()]
user_df.head()

,id,active,email,password,role,temp_pw,time,user_name
0,3,b'\x01',92f8a6bc-c,$2a$10$yG8.Xpcd91BzO0z2zJSwguOih3emckrD.Ouga5R...,None,None,NaT,00c04056-58
1,4,b'\x01',7ce5f453-e,$2a$10$Ge8tRj4hwT1vv.MIkMYqeuKZYrBPxlJG0haZqm0...,None,None,NaT,7c12ec6e-98
2,5,b'\x01',48d7bd60-3,$2a$10$IlTwPARUqUDZYex68XCvh.pZAxaHmc4jppo9N.J...,None,None,NaT,939396d1-ca
3,6,b'\x01',51e62ad4-a,$2a$10$yHdM/J1cL1/qik8MKfo/X.dQAVRWNINvXAmT9lJ...,None,None,NaT,466dcd86-93
4,7,b'\x01',0fed065c-0,$2a$10$tVljIbO6PHRwhPMhGYHABuih.aidkf6X90vl/lK...,None,None,NaT,449874b1-94


In [6]:
rsdf = Random_Weighted_Sampling(menu_df,user_df,10000)

In [7]:
rsdf

,menu_id,time,user_name,cleaning,couple,drink,family,friend,number,plating,price,restaurant_id,service,solo,flavor,url,id
0,30,2023-12-08 15:36:37.849435,1b6ad635-87,3.0,NaN,NaN,NaN,NaN,1,2.0,3.0,1323716207,NaN,NaN,1,https://youngcheline.s3.ap-northeast-2.amazona...,1
1,30,2023-12-08 15:36:37.849435,d9ebf3ae-ee,1.0,NaN,NaN,NaN,NaN,2,NaN,3.0,1323716207,1.0,NaN,3,https://youngcheline.s3.ap-northeast-2.amazona...,1
2,16,2023-12-08 15:36:37.853972,ac24d447-52,NaN,0.0,3.0,NaN,NaN,3,NaN,1.0,14541884,2.0,NaN,1,https://youngcheline.s3.ap-northeast-2.amazona...,1
3,21,2023-12-08 15:36:37.853972,a508092c-0f,1.0,NaN,NaN,NaN,NaN,4,NaN,NaN,983114429,2.0,NaN,2,https://youngcheline.s3.ap-northeast-2.amazona...,1
4,10,2023-12-08 15:36:37.858423,d451e7a2-80,1.0,NaN,NaN,NaN,NaN,5,NaN,2.0,20638851,NaN,2.0,1,https://youngcheline.s3.ap-northeast-2.amazona...,1
...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...
9995,31,2023-12-08 15:36:46.609286,f57a5676-61,3.0,NaN,NaN,NaN,NaN,9996,1.0,3.0,277055458,1.0,NaN,1,https://youngcheline.s3.ap-northeast-2.amazona...,1
9996,29,2023-12-08 15:36:46.609286,6db6befa-0b,2.0,0.0,NaN,1.0,NaN,9997,NaN,4.0,1032781993,NaN,NaN,1,https://youngcheline.s3.ap-northeast-2.amazona...,4
9997,11,2023-12-08 15:36:46.609286,bcaef27e-4a,1.0,NaN,3.0,NaN,4.0,9998,NaN,1.0,1258680088,1.0,NaN,3,https://youngcheline.s3.ap-northeast-2.amazona...,3
9998,32,2023-12-08 15:36:46.609286,26344058-fd,3.0,NaN,NaN,NaN,NaN,9999,3.0,NaN,777963780,1.0,NaN,1,https://youngcheline.s3.ap-northeast-2.amazona...,2
